In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### This part covered in future section

In [4]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [5]:
train_df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')

In [6]:
train_df.head()

,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0
2,101_00.png,0,0
3,102_00.png,0,0
4,103_00.png,0,0


In [34]:
train_images = os.getcwd() + '/datasets/train_images/' + train_df.iloc[:, 0].values
test_images = './datasets/test_images/' + test_df.iloc[:, 0].values

In [35]:
train_labels = train_df.iloc[:,1:].values
test_labels = test_df.iloc[:,1:].values

In [36]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [37]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

#### Actual focus of this section

In [11]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
output1 = layers.Dense(10, activation="softmax", name="first_num")(x) # First branch
output2 = layers.Dense(10, activation="softmax", name="second_num")(x) # Second branch

model = keras.Model(inputs=inputs, outputs=[output1, output2]) # This is where functional API is useful but not sequential

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 tf.nn.relu (TFOpLambda)        (None, 64, 64, 32)   0           ['batch_normalization[0][0]']

In [13]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(), 
    metrics=["accuracy"],
) # These can also be passed into list for different branch but if only one is passed, it applies for all.

In [14]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 49s 44ms/step - loss: 1.9866 - first_num_loss: 0.9447 - second_num_loss: 0.9611 - first_num_accuracy: 0.6633 - second_num_accuracy: 0.6546
Epoch 2/5
1000/1000 [==============================] - 58s 58ms/step - loss: 0.5276 - first_num_loss: 0.2341 - second_num_loss: 0.2313 - first_num_accuracy: 0.9237 - second_num_accuracy: 0.9236
Epoch 3/5
1000/1000 [==============================] - 83s 83ms/step - loss: 0.3495 - first_num_loss: 0.1525 - second_num_loss: 0.1422 - first_num_accuracy: 0.9503 - second_num_accuracy: 0.9535
Epoch 4/5
1000/1000 [==============================] - 89s 89ms/step - loss: 0.2752 - first_num_loss: 0.1171 - second_num_loss: 0.1075 - first_num_accuracy: 0.9630 - second_num_accuracy: 0.9649
Epoch 5/5
1000/1000 [==============================] - 95s 94ms/step - loss: 0.2281 - first_num_loss: 0.0934 - second_num_loss: 0.0869 - first_num_accuracy: 0.9706 - second_num_accuracy: 0.9725


In [38]:
model.evaluate(test_dataset)

313/313 [==============================] - 19s 59ms/step - loss: 1.0645 - first_num_loss: 0.3539 - second_num_loss: 0.6651 - first_num_accuracy: 0.9037 - second_num_accuracy: 0.8232


[1.064477801322937,
 0.35389256477355957,
 0.6651206612586975,
 0.9036999940872192,
 0.823199987411499]